In [3]:
from pymol import cmd
import py3Dmol

from vina import Vina

from openbabel import pybel

from pdbfixer import PDBFixer
from simtk.openmm.app import PDBFile

from rdkit import Chem
from rdkit.Chem import AllChem,rdFMCS, Draw

import MDAnalysis as mda
from MDAnalysis.coordinates import PDBQT

import prolif as plf
from prolif.plotting.network import LigNetwork
    
%config Completer.use_jedi = False

In [ ]:
#!wget http://mgltools.scripps.edu/downloads/previous-releases/downloads/tars/releases/nightly/1.0/REL/NewLinux/ADFRsuite_x86_64Linux_1.0.tar.gz
#!tar zxf ADFRsuite_x86_64Linux_1.0.tar.gz
#!mkdir ADFRsuite
#!cd ADFRsuite_x86_64Linux_1.0
#!/install.sh -d ../ADFRSuite -c 0
#!export PATH=/mnt/c/Users/angel/Desktop/Reverse_docking_vina/ADFRsuite/bin:$PATH

In [ ]:
cmd.fetch(code='1IEP',type='pdb1')
cmd.select(name='Prot',selection='polymer.protein')
cmd.h_add(selection='all')
cmd.select(name='Lig',selection='organic')
cmd.save(filename='1AZ8_clean.pdb',format='pdb',selection='Prot')
cmd.save(filename='1AZ8_lig.mol2',format='mol2',selection='Lig')

In [ ]:
!ADFRSuite/bin/prepare_receptor -v -r 1AZ8_clean.pdb -A hydrogens -o 1AZ8_clean.pdbqt

In [ ]:
m=Chem.MolFromMol2File('1AZ8_lig.mol2',False)
Draw.MolToImage(m)

In [ ]:
mol= [m for m in pybel.readfile(filename='1AZ8_lig.mol2',format='mol2')][0]
mol.addh()
out=pybel.Outputfile(filename='1AZ8_lig.mol2',format='mol2',overwrite=True)
out.write(mol)
out.close()

In [ ]:
m=Chem.MolFromMol2File('1AZ8_lig.mol2')
m

In [ ]:
!ADFRSuite/bin/prepare_ligand -v -l 1AZ8_lig.mol2 -o 1AZ8_lig.pdbqt

In [ ]:
view = py3Dmol.view()
view.removeAllModels()
view.setViewStyle({'style':'outline','color':'black','width':0.2})

view.addModel(open('1AZ8_clean.pdb','r').read(),'pdb')
Prot=view.getModel()
Prot.setStyle({'cartoon':{'arrows':True, 'tubes':True, 'style':'oval', 'color':'white'}})
view.addSurface(py3Dmol.VDW,{'opacity':0.6,'color':'white'})

view.addModel(open('1AZ8_lig.mol2','r').read(),'mol2')
IN4 = view.getModel()
IN4.setStyle({},{'stick':{'colorscheme':'greenCarbon','radius':0.1}})

view.zoomTo()
view.show()

In [ ]:
def getbox(selection = "sele", extending = 6.0, software='vina'):                                                                                             
    ([minX, minY, minZ],[maxX, maxY, maxZ]) = cmd.get_extent(selection)
    minX = minX - float(extending)
    minY = minY - float(extending)
    minZ = minZ - float(extending)
    maxX = maxX + float(extending)
    maxY = maxY + float(extending)
    maxZ = maxZ + float(extending)
    
    SizeX = maxX - minX
    SizeY = maxY - minY
    SizeZ = maxZ - minZ
    CenterX =  (maxX + minX)/2
    CenterY =  (maxY + minY)/2
    CenterZ =  (maxZ + minZ)/2
    if software == 'vina':
        return {'center_x':CenterX,'center_y': CenterY, 'center_z': CenterZ},{'size_x':SizeX,'size_y': SizeY,'size_z': SizeZ}
    elif software == 'ledock':
        return {'minX':minX, 'maxX': maxX},{'minY':minY, 'maxY':maxY}, {'minZ':minZ,'maxZ':maxZ}
    elif software == 'both':
        return ({'center_x':CenterX,'center_y': CenterY, 'center_z': CenterZ},{'size_x':SizeX,'size_y': SizeY,'size_z': SizeZ}),({'minX':minX, 'maxX': maxX},{'minY':minY, 'maxY':maxY}, {'minZ':minZ,'maxZ':maxZ})
    
    else:
        print('software options must be "vina", "ledock" or "both"')

In [ ]:
box_center,box_size=getbox(selection='Lig',extending=5.0,software='vina')
print(box_center,'\n',box_size)

In [ ]:
v = Vina(sf_name='vina')

v.set_receptor('1AZ8_clean.pdbqt')

v.set_ligand_from_file('1AZ8_lig.pdbqt')

v.compute_vina_maps(center=[box_center['center_x'], box_center['center_y'], box_center['center_z']], 
                    box_size=[box_size['size_x'], box_size['size_y'], box_size['size_z']])

'''
# Score the current pose
energy = v.score()
print('Score before minimization: %.3f (kcal/mol)' % energy[0])

# Minimized locally the current pose
energy_minimized = v.optimize()
print('Score after minimization : %.3f (kcal/mol)' % energy_minimized[0])
v.write_pose('1AZ8_ligand_minimized.pdbqt', overwrite=True)
'''

# Dock the ligand
v.dock(exhaustiveness=8, n_poses=10)
v.write_poses('1AZ8_ligand_vina_out.pdbqt', n_poses=5, overwrite=True)

In [ ]:
results = [m for m in pybel.readfile(filename='1AZ8_ligand_vina_out.pdbqt',format='pdbqt')]
out=pybel.Outputfile(filename='1AZ8_ligand_vina_out.sdf',format='sdf',overwrite=True)
for pose in results:
    pose.addh()
    out.write(pose)
out.close()

In [ ]:
results=Chem.SDMolSupplier('1AZ8_ligand_vina_out.sdf')

view = py3Dmol.view()
view.removeAllModels()
view.setViewStyle({'style':'outline','color':'black','width':0.2})

view.addModel(open('1AZ8_clean.pdbqt','r').read(),'pdb')
Prot=view.getModel()
Prot.setStyle({'cartoon':{'arrows':True, 'tubes':True, 'style':'oval', 'color':'white'}})
view.addSurface(py3Dmol.VDW,{'opacity':0.6,'color':'white'})

view.addModel(open('1AZ8_lig.mol2','r').read(),'mol2')
IN4 = view.getModel()
IN4.setStyle({},{'stick':{'colorscheme':'greenCarbon','radius':0.2}})


pose_1=Chem.MolToMolBlock(results[0])
print ('Pose: {} | Score: {} \n'.format(results[0].GetProp('MODEL'),results[0].GetProp('REMARK')))

print('Reference: Green | Docking Pose: Purple')

view.addModel(pose_1,'mol')
x = view.getModel()
x.setStyle({},{'stick':{'colorscheme':'purpleCarbon','radius':0.2}})

view.zoomTo()
view.show()

In [6]:
# load protein
prot = mda.Universe("1AZ8_clean.pdb",guess_bonds=True)
prot = plf.Molecule.from_mda(prot)
prot.n_residues

RDKit ERROR: [01:12:18] Explicit valence for atom # 1132 H, 2, is greater than permitted


AtomValenceException: Explicit valence for atom # 1132 H, 2, is greater than permitted

In [ ]:
# load ligands
path = str('1AZ8_ligand_vina_out.sdf')
lig_suppl = list(plf.sdf_supplier(path))
# generate fingerprint
fp = plf.Fingerprint()
fp.run_from_iterable(lig_suppl, prot)
df = fp.to_dataframe(return_atoms=True)
df

In [ ]:


#fp = plf.Fingerprint()
#fp.run(u.trajectory[::10], lig, prot)
#df = fp.to_dataframe(return_atoms=True)

net = LigNetwork.from_ifp(df,lig_suppl[0],kind="frame", frame=1,rotation=270)
net.display()

In [ ]:
lig_suppl

In [ ]:
!wget http://www.lephar.com/download/lepro_linux_x86
!wget http://www.lephar.com/download/ledock_linux_x86

In [1]:
!./lepro_linux_x86 1AZ8_clean.pdb

In [ ]:
X,Y,Z=getbox(selection='Lig',extending=5.0,software='ledock')

In [ ]:
with open('dock.in') as file:
    for line in file.readlines():
        print(line.strip())